# Importando Bibliotecas

In [19]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support


# Inicializando Dataframes

In [2]:
df = pd.read_csv('exame_cmc13_dados_treinamento.csv', sep=';')
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'user_id', 'age', 'isbn', 'rating',
       'book_title', 'book_author', 'year_of_publication', 'publisher',
       'img_l', 'Language', 'Category', 'city', 'state', 'country'],
      dtype='object')

# Tratando Dados

Vemos campos que não são mencionados e que, provavelmente, se referem apenas a identificações internas dos livros: 'Unnamed: 0.1' e 'Unnamed: 0'. Vamos excluí-los.

In [3]:
df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

Vamos começar a exluir colunas inúteis para a análise. Em um primeiro momento, podemos excluir as colunas de identificação do usuário:

In [4]:
df = df.drop(['user_id'], axis=1)

Agora, vamos filtrar as localidades. Para simplificar a análise, contaremos que leitores de um mesmo país têm gostos semelhantes, excluindo a necessidade de identificadores de cidades e estados. Isso também é permitido pelo fato de haver uma variedade de países:

In [5]:
df['country'].value_counts()

usa                102798
canada              12113
united kingdom       2577
australia            1631
germany              1077
                    ...  
mozambique              1
new jersey, usa         1
ontario, canada         1
iowa, usa               1
haiti                   1
Name: country, Length: 187, dtype: int64

Existem 116 países no dataset. Excluindo colunas referentes a localidades:

In [6]:
df = df.drop(['city', 'state'], axis=1)

Análise da linguagem dos livros:

In [7]:
df['Language'].value_counts()

en    87209
9     43970
Name: Language, dtype: int64

Veja que apenas duas classificações foram categorizadas: en (inglês) e 9 (provavelmente um placeholder ou erro de obtenção de dados). Como a análise desses dois classificadores não nos fornece tanta informação, é razoável excluir tal coluna.

In [8]:
df = df.drop(['Language'], axis=1)

Podemos excluir as colunas 'isbn' porque se refere a uma identificação do livro, redundante com o título, e 'img_l', pois, apesar de a capa certamente ser importante para a escolha de um livro, apenas o link da imagem não adiciona tanto à análise.

In [9]:
df = df.drop(['isbn', 'img_l'], axis=1)

Veja que as colunas de idade e ano de publicação estão sendo tratadas como floats. Convertendo ambas para int:

In [10]:
df['age'] = df['age'].astype(int)
df['year_of_publication'] = df['year_of_publication'].astype(int)

Entretanto, ainda devem existir colunas com valores nulos. Vamos contá-las:

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131179 entries, 0 to 131178
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   age                  131179 non-null  int32 
 1   rating               131179 non-null  int64 
 2   book_title           131179 non-null  object
 3   book_author          131179 non-null  object
 4   year_of_publication  131179 non-null  int32 
 5   publisher            131179 non-null  object
 6   Category             131179 non-null  object
 7   country              126615 non-null  object
dtypes: int32(2), int64(1), object(5)
memory usage: 7.0+ MB


Veja que apenas 126615 linhas de 'country' tem valores não nulos, contra 131179 das outras colunas. Como são apenas 4564 de 130000 dados, representando aproximadamente 3% do dataset, podemos excluí-los sem grande perda:

In [12]:
df = df.dropna()

# Construção do modelo MLP

In [13]:
X_train = df.drop(['rating'], axis = 1)
Y_train = df['rating']

Encodificando as Strings para que o fit no modelo funcione (strings não têm valor de análise para o SKLearn, por isso precisam ser encodificadas):

In [14]:
le = preprocessing.LabelEncoder()
for column_name in X_train.columns:
    if X_train[column_name].dtype == object:
        X_train[column_name] = le.fit_transform(X_train[column_name])
    else:
        pass

Carregando os dados de teste:

In [15]:
df_test = pd.read_csv('exame_cmc13_dados_teste.csv', sep=';')

Aplicando o mesmo tratamento feito aos dados de treino:

In [16]:
df_test = df_test.drop(['Unnamed: 0.1', 'Unnamed: 0', 'user_id', 'city', 'state', 'Language', 'isbn', 'img_l'], axis=1)
df_test['age'] = df_test['age'].astype(int)
df_test['year_of_publication'] = df_test['year_of_publication'].astype(int)

X_test = df_test.drop(['rating'], axis = 1)
Y_test = df_test['rating']

for column_name in X_test.columns:
    if X_test[column_name].dtype == object:
        X_test[column_name] = le.fit_transform(X_test[column_name])
    else:
        pass

Vamos testar qual o número de camadas que otimiza a classificação. Para isso, façamos uma lista com as porcentagens de acerto:

In [17]:
# percebeu-se que não haviam melhorias significativas de precisão após a adição de mais camadas no modelo de 2 hidden layers
# de 100 perceptrons cada.
clf = MLPClassifier(hidden_layer_sizes = (150, 150), max_iter = 400, random_state=2).fit(X_train, Y_train)

Testando a precisão do modelo:

In [18]:
clf.score(X_test, Y_test)

0.589815520658637

Vamos obter um overview completo da precisão, recall e fscore. Começaremos definindo o Y previsto com base no modelo criado:

In [20]:
Y_pred = clf.predict(X_test)

In [28]:
precision_recall_fscore_support(Y_test, Y_pred, average='micro')
# micro porque as métricas são levadas em conta calculando apenas os TN, TP, FN, FP, vide documentação

(0.589815520658637, 0.589815520658637, 0.589815520658637, None)